<a href="https://colab.research.google.com/github/MajdTaweel/cv-project-2/blob/main/Accuracy_measure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install imageai==2.1.5 tensorflow==1.14.0 tensorflow-gpu==1.14.0 keras==2.3.1

     |████████████████████████████████| 184kB 5.3MB/s 
     |████████████████████████████████| 109.2MB 75kB/s 
     |████████████████████████████████| 377.0MB 40kB/s 
     |████████████████████████████████| 378kB 40.7MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 491kB 37.4MB/s 
     |████████████████████████████████| 3.2MB 42.0MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


#import Dataset and models


In [ ]:
!rm -r content/annotations

In [3]:
from google.colab import drive, files
drive.mount('/content/drive')
!cp -r "/content/drive/My Drive/CV Project2/PalestinePlateDataSet/annotations" /content
drive.flush_and_unmount()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!mkdir /content/{models,json}
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/My\ Drive/CV\ Project2/PalestinePlateModels4/*010*.h5 /content/models
!cp /content/drive/My\ Drive/CV\ Project2/PalestinePlateModels4/json/*.json /content/json
drive.flush_and_unmount()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
!git clone https://github.com/sergiomsilva/alpr-unconstrained
!cd alpr-unconstrained/darknet/ && make
!cd alpr-unconstrained && bash get-networks.sh

Cloning into 'alpr-unconstrained'...
remote: Enumerating objects: 290, done.
remote: Total 290 (delta 0), reused 0 (delta 0), pack-reused 290
Receiving objects: 100% (290/290), 2.62 MiB | 13.40 MiB/s, done.
Resolving deltas: 100% (91/91), done.
mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -fopenmp -Ofast -c ./src/gemm.c -o obj/gemm.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -fopenmp -Ofast -c ./src/utils.c -o obj/utils.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -fopenmp -Ofast -c ./src/cuda.c -o obj/cuda.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -fopenmp -Ofast -c ./src/deconvolutional_layer.c -o obj/deconvolutional_layer.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -fopenmp -Ofast -c ./src/convolutional_layer

#split to train and validation


In [6]:
!mkdir /content/validation

In [8]:
import os, random

directory = '/content';

filenames = os.listdir(directory + '/annotations')

indices=[i for i in range(len(filenames))]

random.seed(2021)
random.shuffle(indices)

split = int(0.8 * len(filenames))
filesplits = {
    'train': [filenames[idx] for idx in indices[:split]],
    'validation': [filenames[idx] for idx in indices[split:]]
}

In [13]:
print(len(filesplits['train']))
print(len(filesplits['validation']))
print(filesplits['validation'][0])

1485
372
img00745.txt


#start process

##plate detector model

In [16]:
from imageai.Detection.Custom import CustomObjectDetection
plateDetector = CustomObjectDetection()
plateDetector.setModelTypeAsYOLOv3()
plateDetector.setModelPath("/content/models/detection_model-ex-010--loss-0003.074.h5")
plateDetector.setJsonPath("/content/json/detection_config.json")
plateDetector.loadModel()

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [17]:
from google.colab import drive, files
drive.mount('/content/drive')
!mkdir platesDetect plates

Mounted at /content/drive


In [36]:
!rm {plates,platesDetect}/*

In [37]:
from PIL import Image

imgDirectory = '/content/drive/My Drive/CV Project2/PalestinePlateDataSet/images'

i=0
vehicles = {}

for car in filesplits['validation']:
  # if(i==3):
  #   break
  i+=1
  print(i)

  tmp = car.split(".")[0]
  imageName = f'{tmp}.jpg'

  detections = plateDetector.detectObjectsFromImage(
      input_image=f'{imgDirectory}/{imageName}',
      output_image_path=f'platesDetect/{tmp}Out.jpg',
      minimum_percentage_probability=20
    )
  
  with open(f'/content/annotations/{car}', 'r') as f:
    lines = f.readlines()
    realNumber = lines[0].strip()


  vehicles[imageName]= {
      "real": realNumber,
      "predicted": ""
  }
  # vehicles[car]['plates'] = {}

  if len(detections) > 0:
    max_prob = max(detections, key=lambda detection: detection['percentage_probability'])
    print(max_prob["name"], " : ", max_prob["percentage_probability"], " : ", max_prob["box_points"])

    carImg = Image.open(f'{imgDirectory}/{imageName}')

    plate = carImg.crop(max_prob["box_points"])
    
    # platename = f'{car.split(".")[0]}_lp'
    plate.save(f'plates/{tmp}-lp.png')


1
plate  :  53.75346541404724  :  [324, 216, 425, 240]
2
plate  :  55.26583194732666  :  [361, 207, 455, 233]
3
plate  :  34.11563038825989  :  [300, 153, 387, 179]
4
plate  :  60.69805026054382  :  [300, 509, 412, 551]
5
plate  :  67.45464205741882  :  [280, 596, 420, 632]
6
plate  :  47.08118140697479  :  [176, 14, 260, 37]
7
plate  :  62.282270193099976  :  [322, 525, 443, 573]
8


/usr/local/lib/python3.6/dist-packages/imageai/Detection/Custom/__init__.py:1234: RuntimeWarning: overflow encountered in exp
  return 1. / (1. + np.exp(-x))


plate  :  54.343181848526  :  [186, 287, 276, 313]
9
plate  :  59.31962728500366  :  [260, 306, 369, 330]
10
plate  :  63.65360617637634  :  [220, 378, 364, 424]
11
plate  :  51.83002948760986  :  [290, 417, 429, 466]
12
plate  :  46.22986912727356  :  [437, 627, 574, 667]
13
plate  :  34.63159799575806  :  [116, 234, 173, 270]
14
plate  :  68.29408407211304  :  [161, 312, 281, 350]
15
plate  :  61.71942949295044  :  [280, 387, 388, 422]
16
plate  :  72.3814606666565  :  [82, 109, 167, 128]
17
plate  :  72.37634062767029  :  [208, 100, 300, 121]
18
plate  :  71.71146869659424  :  [228, 408, 366, 454]
19
plate  :  72.11154699325562  :  [288, 481, 398, 510]
20
plate  :  52.68322825431824  :  [244, 307, 336, 337]
21
plate  :  52.73414850234985  :  [211, 240, 304, 271]
22
plate  :  62.85848617553711  :  [178, 364, 315, 408]
23
plate  :  61.57795786857605  :  [142, 278, 245, 314]
24
plate  :  61.67657971382141  :  [262, 391, 370, 425]
25
plate  :  60.18732786178589  :  [370, 527, 508, 573]


In [77]:
vehicles

{'img00006.jpg': {'predicted': '6309994', 'ratio': 1.0, 'real': '6309994'},
 'img00014.jpg': {'predicted': '1316786',
  'ratio': 0.8571428571428571,
  'real': '3167864'},
 'img00021.jpg': {'predicted': '12537H', 'ratio': 1.0, 'real': '12537H'},
 'img00024.jpg': {'predicted': '6073393', 'ratio': 1.0, 'real': '6073393'},
 'img00025.jpg': {'predicted': '7762195', 'ratio': 1.0, 'real': '7762195'},
 'img00026.jpg': {'predicted': '6300999', 'ratio': 1.0, 'real': '6300999'},
 'img00028.jpg': {'predicted': '6195931', 'ratio': 1.0, 'real': '6195931'},
 'img00032.jpg': {'predicted': 'I616993',
  'ratio': 0.8571428571428571,
  'real': '6169930'},
 'img00034.jpg': {'predicted': '6169930', 'ratio': 1.0, 'real': '6169930'},
 'img00038.jpg': {'predicted': '6283730', 'ratio': 1.0, 'real': '6283730'},
 'img00039.jpg': {'predicted': 'G633795',
  'ratio': 0.8571428571428571,
  'real': '6633795'},
 'img00042.jpg': {'predicted': 'I612713',
  'ratio': 0.8571428571428571,
  'real': '6127130'},
 'img00044.jpg

In [44]:
for i,key in enumerate(vehicles):
  if(i==1):
    break
  print(key)
  print("predicted",vehicles[key]['predicted'])
  print("real",vehicles[key]['real'])

img00745.jpg
predicted 
real 6844793


In [42]:
!ls plates | wc -l
!ls platesDetect | wc -l

372
372


In [ ]:
!cd alpr-unconstrained/ && python2 license-plate-ocr.py /content/plates

In [63]:
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [87]:
correct =0
wrong =0
ratioSum =0

for i,key in enumerate(vehicles):
  # if(i==1):
  #   break
  try:
    with open(f'/content/plates/{key.split(".")[0]}-lp_str.txt', 'r') as f:
      vehicles[key]["predicted"] = f.readlines()[0].strip()[:7]
    vehicles[key]["ratio"] = similar(vehicles[key]["real"].upper(),vehicles[key]["predicted"])
    ratioSum += vehicles[key]["ratio"]
    if(vehicles[key]["ratio"] == 1.0):
      correct += 1
    else:
      wrong += 1
  except:
    vehicles[key]["ratio"]=0
    wrong +=1

print("Plate Accuracy: ", "1.0")  
print("OCR Accurace: => ",correct , "/", wrong, " =  ", (correct/(correct+wrong))*100 )
print("percentage: => ", ratioSum/len(vehicles))

  


Plate Accuracy:  1.0
OCR Accurace: =>  225 / 147  =   60.483870967741936
percentage: =>  0.9059486809486802


In [86]:
for key in vehicles:
  if(vehicles[key]["ratio"] != 1.0):
    print(vehicles[key])

{'real': '6067895', 'predicted': '6067095', 'ratio': 0.8571428571428571}
{'real': '6013097', 'predicted': '60T3097', 'ratio': 0.8571428571428571}
{'real': '6127130', 'predicted': 'I612713', 'ratio': 0.8571428571428571}
{'real': '6800298', 'predicted': 'B800298', 'ratio': 0.8571428571428571}
{'real': '6231730', 'predicted': '623T730', 'ratio': 0.8571428571428571}
{'real': '7270294', 'predicted': 'I727029', 'ratio': 0.8571428571428571}
{'real': '6219230', 'predicted': '621923U', 'ratio': 0.8571428571428571}
{'real': '6119593', 'predicted': '619533', 'ratio': 0.7692307692307693}
{'real': '7526796', 'predicted': '752679G', 'ratio': 0.8571428571428571}
{'real': '6331694', 'predicted': 'B331649', 'ratio': 0.7142857142857143}
{'real': '7197696', 'predicted': 'I7197G9', 'ratio': 0.7142857142857143}
{'real': '6175930', 'predicted': 'IG17593', 'ratio': 0.7142857142857143}
{'real': '13265h', 'predicted': '132656H', 'ratio': 0.9230769230769231}
{'real': '6000298', 'predicted': 'I600D29', 'ratio': 